In [15]:
from meep_funcs import *
from gen_geo.bounded_voronoi import *
import sys
from meep_funcs import *
from scipy.integrate import romb
np.random.seed(15)



# dist = float(sys.argv[1])
# geo = create_simple_geo(dist)
# vor = b_voronoi(120)
v_seed_points = np.random.rand(800, 3) - 0.5
vor = Voronoi(v_seed_points)
my_voronoi_geo = voronoi_geo(100, vor = vor)

# sim = create_sim(mode='eps', arg=my_eps, my_voronoi_geo = my_voronoi_geo, res = 50)
# get_sim_output(str(sys.argv[2]), sim, length_t = 400, out_every=0.6, get_3_field=False)

# out_num_geo('checker_geo.bin', my_checker_geo, range_geo=[1.0,1.0,1.0], range_index=[100,100,100])


In [23]:
v_seed_points = np.random.rand(3000, 3) - 0.5
vor = Voronoi(v_seed_points)
my_voronoi_geo = voronoi_geo(100, vor = vor)

def absorbing(index):
    coord = index2coord(np.array(index), [100, 100, 100], [1, 1, 1])
    ass = my_ass(my_voronoi_geo, coord)
    if ass == 1:
        return True
    else:
        return False

def clean_arr(arr):
    only_absorb_array = np.zeros(arr.shape)
    for x in range(arr.shape[1]):
        for y in range(arr.shape[2]):
            for z in range(arr.shape[3]):
                if absorbing((x, y, z)):
                    only_absorb_array[:, x, y, z] = arr[:, x, y, z] 
    return only_absorb_array

def get_ms(arr, rms_interval):
    rms_shape = np.array(arr.shape)
    rms_shape[0] = np.floor(rms_shape[0]/rms_interval)
    rms_arr = np.zeros(rms_shape.astype(np.int))
    for i in range(0, rms_shape[0]):
        slice = arr[i*rms_interval:(i+1)*rms_interval]
        slice = slice**2
        int_slice = romb(slice, dx=1, axis=0)/(rms_interval-1)
        # rms_arr[i, :, :, :] = np.sqrt(np.mean(slice, axis=0))
        rms_arr[i, :, :, :] = int_slice
    return rms_arr

add_subfix = lambda name, sub_fix : file_name[0:-4] + sub_fix + file_name[-4:]

file_name = 'meep_out/voronoi_120_t_400_res_50.bin'
arr = read_windows(file_name)


In [24]:
# file_name = add_subfix(file_name, '_rms_' + str(rms_interval))
file_name = add_subfix(file_name, '_clean')

rms_interval = 5
clean_arr = clean_arr(arr)
rms_arr = get_ms(clean_arr, 5)
write_windows(rms_arr, file_name)


In [35]:
rms_arr.shape

(133, 100, 100, 100)

In [10]:
romb([1,2,3,4,4])

11.688888888888888